In [2]:
import pandas as pd
import pickle as pkl

In [6]:
with open('testing.pkl', 'rb') as file:
    x = pkl.load(file)

with open('testingii.pkl', 'rb') as file:
    y = pkl.load(file)

In [25]:
y.isna().sum().sort_values()

play_type                          0
pitcher_season_intent_walk         0
pitcher_season_double_play         0
pitcher_season_sacrifice           0
pitcher_season_single              0
                               ...  
season_error                       0
temprature_sq                      0
on_1b                         129151
on_2b                         148848
on_3b                         165766
Length: 72, dtype: int64

In [28]:
# Precompute last month and last season league averages for each pitbat combo
league_averages_list = []
from datetime import timedelta
import constants

# Iterate over each pitbat combo
for pitbat_combo in constants.HAND_COMBOS:
    pitbat_df = y[y.pitbat == pitbat_combo].copy()

    # Iterate over unique dates
    for date in pitbat_df.game_date.unique():
        season_ago = date - timedelta(days=365)  # 1 year ago
        month_ago = date - timedelta(days=30)    # 1 month ago
        
        # Filter data within the last month and season
        season_pitbat_date_df = pitbat_df[(pitbat_df.game_date < date) & (pitbat_df.game_date >= season_ago)]
        month_pitbat_date_df = pitbat_df[(pitbat_df.game_date < date) & (pitbat_df.game_date >= month_ago)]
        
        # Precompute league averages for each play type and store them in a list
        for play in constants.PLAY_TYPES:
            season_play_average = len(season_pitbat_date_df[season_pitbat_date_df.play_type == play]) / len(season_pitbat_date_df) if date >= season_pitbat_date_df.game_date.min() + timedelta(days=364) > 0 else 1
            month_play_average = len(month_pitbat_date_df[month_pitbat_date_df.play_type == play]) / len(month_pitbat_date_df) if date >= month_pitbat_date_df.game_date.min() + timedelta(days=29) else 1
            
            league_averages_list.append({
                "pitbat": pitbat_combo,
                "game_date": date,
                "play_type": play,
                f"season_{play}_average": season_play_average,
                f"month_{play}_average": month_play_average
            })

    # Create a DataFrame from the list of league averages
    league_averages_df = pd.DataFrame(league_averages_list)

play_type                         0
pitcher_season_intent_walk        0
pitcher_season_double_play        0
pitcher_season_sacrifice          0
pitcher_season_single             0
                              ...  
season_error                      0
temprature_sq                     0
on_1b                         51278
on_2b                         58830
on_3b                         65825
Length: 72, dtype: int64
play_type                         0
pitcher_season_intent_walk        0
pitcher_season_double_play        0
pitcher_season_sacrifice          0
pitcher_season_single             0
                              ...  
season_error                      0
temprature_sq                     0
on_1b                         24048
on_2b                         28229
on_3b                         31213
Length: 72, dtype: int64
play_type                         0
pitcher_season_intent_walk        0
pitcher_season_double_play        0
pitcher_season_sacrifice          0
pitcher_season

In [31]:
month_pitbat_date_df

,play_type,game_pk,game_date,ballpark,wind_speed,wind_direction,batter,pitcher,pitbat,on_3b,...,pitcher_month_double_play,pitcher_month_intent_walk,pitcher_month_triple,pitbat_y,Left to Right,Right to Left,in,out,zero,temprature_sq
182339,double,448885,2016-09-03,Globe Life II,8,out,429664,572140,LL,<NA>,...,0.000000,0.0,0.000122,RR,0,0,0,8,0,4356
182340,walk,448885,2016-09-03,Coors,8,out,592325,572140,LL,<NA>,...,0.000000,0.0,0.000122,RR,0,0,0,8,0,4356
182341,field_out,448885,2016-09-03,Truist,8,out,572122,572140,LL,429664,...,0.000000,0.0,0.000122,RR,0,0,0,8,0,4356
182342,field_out,448891,2016-09-03,Progressive,12,Right to Left,458015,448802,LL,<NA>,...,0.000000,0.0,0.000122,RR,0,12,0,0,0,6241
182343,home_run,448882,2016-09-03,Minute Maid,17,Right to Left,594809,502327,LL,<NA>,...,0.000000,0.0,0.000122,RR,0,17,0,0,0,5476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184451,field_out,449279,2016-10-01,Minute Maid,11,Left to Right,570256,543779,LL,<NA>,...,0.000064,0.0,0.000000,RR,11,0,0,0,0,4489
184452,strikeout,449280,2016-10-01,Comerica,0,zero,543105,643355,LL,<NA>,...,0.000064,0.0,0.000000,RR,0,0,0,0,0,5625
184453,single,449279,2016-10-01,Comerica,11,Left to Right,459964,543779,LL,<NA>,...,0.000064,0.0,0.000000,RR,11,0,0,0,0,4489
184454,field_out,449281,2016-10-01,Rodgers,5,in,572816,502706,LL,<NA>,...,0.000064,0.0,0.000000,RR,0,0,5,0,0,6241
